## 종목별 연도별 수익률 카테고리화 작업

In [2]:
import FinanceDataReader as fdr
fdr.__version__

'0.6.0'

In [3]:
# import FinanceDataReader as fdr

# 한국거래소 상장종목 전체
df_krx = fdr.StockListing('KRX')
df_krx.head()

,Symbol,Name,Sector,Industry
0,001040,CJ,기타 금융업,지주회사
1,011150,CJ씨푸드,기타 식품 제조업,"수산물(어묵,맛살)가공품 도매,원양수산업,수출입"
2,012630,HDC,건물 건설업,"토목공사,건축공사,아파트분양사업,재개발/재건축사업"
3,082740,HSD엔진,일반 목적용 기계 제조업,"대형선박용엔진,내연발전엔진"
4,001390,KG케미칼,기초 화학물질 제조업,"콘크리트혼화제, 비료, 친환경농자재, 수처리제"


In [4]:
import numpy as np 
from statsmodels import regression 
import statsmodels.api as sm 
import matplotlib.pyplot as plt 
import math 
import pandas as pd 
import pandas_datareader as pdr

In [5]:
def stock_select(code):

    stock_df = fdr.DataReader(code,'2012','2018-09-30')

    # Date를 인덱스에서 제거하고 항목으로 사용
    stock_df = stock_df.reset_index()

    # Date에서 연도를 추출함
    stock_df['Year'] = stock_df.Date.map(lambda x : x.year )
    stock_df['Code'] = code

    return stock_df


### 수익률 계산

In [6]:
# [참고]
# 일별 수익률은 FinanceDataReader에서 제공하고 있으므로 Skip
# rets = np.log(data / data.shift(1))
# rets.mean() * 252

### 연간수익률 만들기

In [7]:
# [참고] 일별 단순평균. 참고만 
stock_df = stock_select('005930')
data_pivot=stock_df.pivot_table('Change',index='Code',columns='Year', aggfunc='mean')
data_pivot

Year,2012,2013,2014,2015,2016,2017,2018
Code,,,,,,,
005930,0.001652,-0.000292,-0.000013,-0.000061,5.589151,1.351477,-0.000324


In [8]:
# 기하평균 함수
def year_change_cal(stock_df):
    stock_df.sort_values(by='Date', ascending=True, inplace=True) # 오름차순 정렬
    
    year_change = 0
    
    # 일별 loop를 통해 수익률 누적
    for x in range(0,len(stock_df)):
        year_change = (1+year_change) * (1+stock_df.Change.iloc[x])
        
    # 최종 연간 누적수익률 계산
    year_change = ( year_change ** (1/len(stock_df))) - 1
    return year_change

In [9]:
# 1개 종목만 추출해보기
year_change_df = pd.DataFrame()
year_change_dict = {}
year_change_dict['Code'] = '005930'

for year in range(2012,2019):
    target_df = stock_df[stock_df.Year==year]
    year_change = year_change_cal(target_df)
    year_change_dict[str(year)] = year_change

    
year_change_df = pd.Series(year_change_dict).to_frame()
year_change_df = year_change_df.T

# 마지막해 수익률 연율화
# 위에서 2018 3분기까지 가져옴. 1년치로 전환
# 0.0264496,  (1 + 0.0264496)^(4/3)-1
year_change_df['2018'] = year_change_df['2018'].map(lambda x : (1+x)**(4/3)-1)

year_change_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Code,2012,2013,2014,2015,2016,2017,2018
0,005930,0.0232963,0.0219638,0.0195141,0.0194912,0.0185498,0.0227427,0.035421


## 상승하락 강도구하기! 
연도별 상승 하락 일수로 2차 미분 시행

In [10]:
import numpy as np

def updown_poly(up_days,down_days):
    
    # 3차 다항식생성
    x = list(range(1,len(up_days)+1))
    
    up_p3 = np.polyfit(x,up_days,3)
    x = list(range(1,len(down_days)+1))
    down_p3 = np.polyfit(x,down_days,3)
    return up_p3, down_p3



## 심볼릭 연산 (미분구하기)
https://datascienceschool.net/view-notebook/5f5f4a966f9042efa43d711008df9c0b/

In [11]:
# 출처: http://codepractice.tistory.com/76 [코딩 연습]
# Juypter 노트북에서 수학식의 LaTeX 표현을 위해 필요함

from sympy import symbols, Limit
from sympy import Derivative
# sympy.init_printing(use_latex='mathjax')

def updown_strength(up_days, down_days, x_value = len(up_days)):
    
    # 함수 생성
    up_m, down_m = updown_poly(up_days, down_days)

    x, a, h = symbols('x, a, h')

    # 함수 정의
    up_fx = up_m[0] * x ** 3 + up_m[1] * x ** 2 + up_m[2] * x + up_m[3]
    down_fx = down_m[0] * x ** 3 + down_m[1] * x ** 2 + down_m[2] * x + down_m[3]


    # up_미분 -------------------------

    # 1차 도함수
    up_d1 = Derivative(up_fx, x).doit()
    # 2차 도함수
    up_d2 = Derivative(up_fx, x, 2).doit()

    # 미분계수
    up_d1_coef = up_d1.subs({x:x_value}) # x 숫자 지정
    up_d2_coef = up_d2.subs({x:x_value}) # x 숫자 지정

    print("up함수 : ", up_d1_coef,up_d2_coef)

    # down_미분 -------------------------

    # 1차 도함수
    down_d1 = Derivative(down_fx, x).doit()
    # 2차 도함수
    down_d2 = Derivative(down_fx, x, 2).doit()

    # 미분계수
    down_d1_coef = down_d1.subs({x:5}) # x 숫자 지정
    down_d2_coef = down_d2.subs({x:5}) # x 숫자 지정


    print("down함수 : ", down_d1_coef,down_d2_coef)
    
    return up_d1_coef, up_d2_coef, down_d1_coef, down_d2_coef

## 상장종목 전체 수익률&상승하락일수 구하기

In [42]:

year_change_df = pd.DataFrame()
year_days_df = pd.DataFrame()
    
    
for x in range(0,len(df_krx)):
    # 종목 데이터 추출
    code = df_krx.Symbol.iloc[x]
    name = df_krx.Name.iloc[x]
    try:
        stock_df = stock_select(code)
    except:
        print(code, name, "종목정보 추출 실패")
        continue
    
    # 초기화
    temp_change_df = pd.DataFrame()
    year_change_dict = {}
    year_change_dict['Code'] = code
    year_change_dict['Name'] = name
    # 일수체크 관련 변수
    temp_days_df = pd.DataFrame()
    year_days_dict = {}
    year_days_dict['Code'] = code
    year_days_dict['Name'] = name    
    up_days_list_17 = []
    down_days_list_17 = []
    tot_days_list_17 = []
    up_days_list_16 = []
    down_days_list_16 = []
    tot_days_list_16 = []
    
    # 연도별 수익률 계산
    for year in range(2012,2019):
        target_df = stock_df[stock_df['Year'] == year]
        
        try:
            # 연간 기하평균 수익률
            year_change = year_change_cal(target_df)
            year_change_dict[str(year)] = year_change
            
            # 상승/하락일 카운팅
            up_days = len(target_df.query("Change >= 0.01")) 
            down_days = len(target_df.query("Change < -0.01")) 
            tot_days = len(target_df)
            
            year_days_dict["up_" + str(year)] = up_days
            year_days_dict["down_" + str(year)] = down_days
            year_days_dict["tot_" + str(year)] = tot_days
            
            if year <= 2017:  # 분석대상기준이 필요하므로
                up_days_list_17.append(up_days)
                down_days_list_17.append(down_days)
                tot_days_list_17 = tot_days_list.append(tot_days)
            
            if year <= 2016: # 분석대상기준이 필요하므로
                up_days_list_16.append(up_days)
                down_days_list_16.append(down_days)
                tot_days_list_16 = tot_days_list.append(tot_days)
            
        except:
            year_change = ""            


    try:
        up_coef1_17, up_coef2_17, down_coef1_17, down_coef2_17 = updown_strength(up_days_list_17,down_days_list_17, len(up_days_list_17)) # 마지막해의 값 2018
        up_coef1_16, up_coef2_16, down_coef1_16, down_coef2_16 = updown_strength(up_days_list_16,up_days_list_16, len(up_days_list_16))
    except:
        print("updown_strength 계산 오류")
        up_coef1 = ""
        up_coef2 = ""
        down_coef1 = ""
        down_coef2 = "" 

    temp_change_df = pd.Series(year_change_dict).to_frame()
    temp_change_df = temp_change_df.T
    temp_days_df = pd.Series(year_days_dict).to_frame()
    temp_days_df = temp_days_df.T
    temp_coef_df = 



    try:
        # 마지막해 수익률 연율화
        # 위에서 2018 3분기까지 가져옴. 1년치로 전환
        # 0.0264496,  (1 + 0.0264496)^(4/3)-1
        temp_change_df['2018'] = temp_change_df['2018'].map(lambda x : (1+x)**(4/3)-1)
        year_change_df = pd.concat([year_change_df, temp_change_df]) # 이전 데이터와 연결
        print(x, "번 : ", code, name, "수익률 생성완료")
    except:
        print(x, "번 : ", code,name, "수익률 생성실패")
        break

    try:
        year_days_df = pd.concat([year_days_df, temp_days_df]) # 이전 데이터와 연결
        print(x, "번 : ", code, name, "Days 생성완료")
    except:
        print(x, "번 : ", code,name, "Days 생성실패")
        break
    

year_change_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


[1, 2, 3, 4, 5, 6, 7]
up함수 :  -11.0753968253968 -6.07142857142858
down함수 :  -5.72222222222219 -6.64285714285715
0 번 :  001040 CJ 수익률 생성완료
0 번 :  001040 CJ Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  0.817460317460316 -10.1190476190476
down함수 :  -1.04761904761902 -8.19047619047619
1 번 :  011150 CJ씨푸드 수익률 생성완료
1 번 :  011150 CJ씨푸드 Days 생성완료
012630 HDC 종목정보 추출 실패
[1, 2, 3, 4, 5, 6, 7]
up함수 :  6.35317460317458 -6.73809523809522
down함수 :  0.571428571428484 -15.6666666666666
3 번 :  082740 HSD엔진 수익률 생성완료
3 번 :  082740 HSD엔진 Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -4.44841269841260 5.35714285714284
down함수 :  -4.45634920634916 3.99999999999998
4 번 :  001390 KG케미칼 수익률 생성완료
4 번 :  001390 KG케미칼 Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -0.424603174603163 -9.85714285714284
down함수 :  -2.90873015873015 -5.00000000000001
5 번 :  010060 OCI 수익률 생성완료
5 번 :  010060 OCI Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -1.60317460317465 -9.19047619047616
down함수 :  -2.85317460317465 -6.14285714285714
6 번 :  002360 SH에너지화학 수

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RankWarning: Polyfit may be poorly conditioned
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



[1, 2, 3, 4, 5, 6, 7]
up함수 :  -7.53571428571424 -4.16666666666670
down함수 :  1.80158730158732 -5.35714285714285
13 번 :  002240 고려제강 수익률 생성완료
13 번 :  002240 고려제강 Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  1.01984126984127 -4.04761904761903
down함수 :  -1.06349206349205 -9.52380952380951
14 번 :  001290 골든브릿지증권 수익률 생성완료
14 번 :  001290 골든브릿지증권 Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -4.64285714285711 -2.35714285714287
down함수 :  -4.06349206349203 -3.69047619047620
15 번 :  014530 극동유화 수익률 생성완료
15 번 :  014530 극동유화 Days 생성완료
214330 금호에이치티 종목정보 추출 실패
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -15.8730158730158 9.64285714285711
down함수 :  -13.5912698412698 9.52380952380948
17 번 :  013700 까뮤이앤씨 수익률 생성완료
17 번 :  013700 까뮤이앤씨 Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -11.1349206349206 0.523809523809511
down함수 :  -10.8253968253968 -7.97619047619047
18 번 :  090350 노루페인트 수익률 생성완료
18 번 :  090350 노루페인트 Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -2.29365079365083 -12.7857142857143
down함수 :  -1.35714285714287 -8.52380952380953
19

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RankWarning: Polyfit may be poorly conditioned
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.


...

[1, 2, 3, 4, 5, 6, 7]
up함수 :  -4.63888888888888 -2.54761904761904
down함수 :  -2.19047619047621 -1.57142857142856
94 번 :  004560 현대비앤지스틸 수익률 생성완료
94 번 :  004560 현대비앤지스틸 Days 생성완료
[1, 2, 3, 4, 5, 6, 7]
up함수 :  -0.285714285714334 -4.16666666666665
down함수 :  -1.08730158730167 -8.23809523809521
95 번 :  057050 현대홈쇼핑 수익률 생성완료
95 번 :  057050 현대홈쇼핑 Days 생성완료
[1, 2]
up함수 :  -186.742845826320 -78.5305475789683
down함수 :  -360.358451331815 -144.905436181800
96 번 :  111110 호전실업 수익률 생성완료
96 번 :  111110 호전실업 Days 생성완료


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RankWarning: Polyfit may be poorly conditioned
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.


[1, 2, 3, 4, 5, 6, 7]
up함수 :  -1.23412698412700 -2.71428571428570
down함수 :  -0.793650793650826 -8.26190476190474
97 번 :  008770 호텔신라 수익률 생성완료
97 번 :  008770 호텔신라 Days 생성완료
[1, 2, 3, 4, 5]
up함수 :  -24.3690476190476 -5.14285714285704
down함수 :  -28.0714285714285 -7.28571428571421
98 번 :  051500 CJ프레시웨이 수익률 생성완료
98 번 :  051500 CJ프레시웨이 Days 생성완료
[1, 2, 3, 4, 5]
up함수 :  -6.13095238095235 -1.85714285714278
down함수 :  5.22619047619051 12.5714285714286
99 번 :  037370 EG 수익률 생성완료
99 번 :  037370 EG Days 생성완료
[1, 2, 3, 4, 5]
up함수 :  -18.2857142857143 9.85714285714290
down함수 :  -13.9523809523809 9.85714285714286
100 번 :  130500 GH신소재 수익률 생성완료
100 번 :  130500 GH신소재 Days 생성완료
[1, 2, 3, 4, 5]
up함수 :  -35.8571428571428 -33.4285714285713
down함수 :  17.2500000000000 24.0000000000000
101 번 :  083450 GST 수익률 생성완료
101 번 :  083450 GST Days 생성완료
[1, 2, 3, 4, 5]
up함수 :  3.23809523809518 36.2857142857143
down함수 :  -24.1428571428571 -12.5714285714285
102 번 :  024120 KB오토시스 수익률 생성완료
102 번 :  024120 KB오토시스 Days 생성완료

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RankWarning: Polyfit may be poorly conditioned
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RankWarning: Polyfit may be poorly conditioned
  # Remove the CWD from sys.path while we load stuff.


,2012,2013,2014,2015,2016,2017,2018,Code,Name
0,0.0238309,0.0221622,0.0217195,0.0213729,0.0207847,0.0214806,0.035331,001040,CJ
0,0.0229535,0.0220815,0.0215381,0.0214025,0.0216315,0.0215846,0.036388,011150,CJ씨푸드
0,0.0215987,0.0224308,0.0210204,0.0194923,0.0195561,0.021273,0.040478,082740,HSD엔진
0,0.0236335,0.0227913,0.0213533,0.0204059,0.0224452,0.0233776,0.034644,001390,KG케미칼
0,0.0216813,0.022888,0.0178584,0.0191435,0.0197449,0.0214233,0.032420,010060,OCI
0,0.0240878,0.021193,0.0228402,0.0193525,0.0202813,0.0197813,0.033499,002360,SH에너지화학
0,0.0220158,0.0223868,0.0202847,0.018546,0.0210161,0.0206686,0.035537,001740,SK네트웍스
0,0.0217215,0.012137,0.0248094,0.0292704,0.0213271,0.125925,0.085129,011810,STX
0,0.0225245,0.0163158,0.0518391,0.0309911,0.0217065,0.0255349,0.030505,071970,STX중공업
0,0.0225665,0.0225299,0.0223952,0.0220869,0.0223411,0.0225794,0.037709,024070,WISCOM


In [ ]:
year_days_dict["up_2017"]

In [14]:
year_days_df

,Code,Name,down_2012,down_2013,down_2014,down_2015,down_2016,down_2017,down_2018,tot_2012,...,tot_2016,tot_2017,tot_2018,up_2012,up_2013,up_2014,up_2015,up_2016,up_2017,up_2018
0,001040,CJ,48,79,70,80,84,72,54,248,...,267,261,198,61,74,76,91,62,61,38
0,011150,CJ씨푸드,66,55,43,72,70,39,41,248,...,260,254,196,76,43,41,78,63,36,38
0,001390,KG케미칼,44,50,64,78,44,54,65,248,...,250,260,205,52,60,68,71,51,55,68
0,010060,OCI,76,59,81,90,80,61,66,248,...,283,284,218,55,66,56,81,80,70,52
0,002360,SH에너지화학,89,74,58,85,74,50,50,248,...,282,285,213,99,63,67,78,68,55,40
0,001740,SK네트웍스,78,69,71,85,51,59,46,248,...,275,272,205,68,64,80,70,60,48,37
0,011810,STX,77,113,97,95,112,18,25,248,...,275,39,67,62,80,72,55,91,18,12
0,071970,STX중공업,82,112,45,59,117,60,62,248,...,256,188,211,67,83,32,33,81,47,42
0,024070,WISCOM,9,18,23,27,33,34,33,248,...,246,243,183,14,21,27,23,25,31,28
0,011420,갤럭시아에스엠,90,54,89,79,76,79,58,248,...,257,257,189,67,71,86,79,69,82,47
